In [1]:
from datetime import date
from glob import glob
import json
import math
import os
import sys
import time
import pickle

import gspread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import get_stim_events, get_evoked_traces, find_nearest_ind
from tbd_eeg.tbd_eeg.data_analysis.eeg_preprocessing import qualitycheck_trials
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [4]:
%matplotlib notebook

#### Functions

In [5]:
## Developed in NPX_find_bursts_testing.ipynb, this version is faster and only returns start times and spike counts ##
def find_bursts_indunit(spike_times):
    
    preISIs = np.diff(spike_times)[:-1]
    postISIs = np.diff(spike_times)[1:]
    ## Find starts ##
    bs_inds = np.nonzero((preISIs > 0.1) * (postISIs < 0.004))[0]
    if len(bs_inds) == 0:
        return np.array([]), np.array([])
    
    burst_starts = bs_inds + 1 # +1 corrects for the actual spike ind
    ## Loop through burst starts to find spikes that belong to the burst
    burst_counts = []
    for st_ind in bs_inds:
        spkind = st_ind+1
        bcount = 1
        while (spkind < len(preISIs)) and (preISIs[spkind] < 0.004):
            spkind += 1
            bcount += 1
        burst_counts.append(bcount)
    
    return spike_times[burst_starts], np.array(burst_counts)

In [6]:
def find_closest_region(sunit_info, struct_tree, annot):
    ## Finds a grey matter region above/below an unknown region ##
    Vind = sunit_info.CCF_DV
    vent_sip = struct_tree.get_structures_by_id([annot[sunit_info.CCF_AP, Vind, sunit_info.CCF_ML]])[0]['structure_id_path']
    while not struct_tree.structure_descends_from(vent_sip[-1], 8):
        Vind += 1
        vent_sip = struct_tree.get_structures_by_id([annot[sunit_info.CCF_AP, Vind, sunit_info.CCF_ML]])[0]['structure_id_path']

    Dind = sunit_info.CCF_DV
    dors_sip = struct_tree.get_structures_by_id([annot[sunit_info.CCF_AP, Dind, sunit_info.CCF_ML]])[0]['structure_id_path']
    while not struct_tree.structure_descends_from(dors_sip[-1], 8):
        Dind -= 1
        dors_sip = struct_tree.get_structures_by_id([annot[sunit_info.CCF_AP, Dind, sunit_info.CCF_ML]])[0]['structure_id_path']

    if (Vind - sunit_info.CCF_DV) <= (sunit_info.CCF_DV - Dind):
        return struct_tree.get_structures_by_id([vent_sip[-1]])[0]['acronym']
    elif (Vind - sunit_info.CCF_DV) > (sunit_info.CCF_DV - Dind):
        return struct_tree.get_structures_by_id([dors_sip[-1]])[0]['acronym']

In [7]:
def get_region_from_children(test_id, parent_id, struct_tree):
    try:
        child_ind = np.nonzero([
            struct_tree.structure_descends_from(test_id, x) for x in struct_tree.child_ids([parent_id])[0]
        ])[0][0]
        return struct_tree.get_structures_by_id([struct_tree.child_ids([parent_id])[0][child_ind]])[0]['acronym']
    except:
        return struct_tree.get_structures_by_id([parent_id])[0]['acronym']

In [8]:
def get_parent_region(region_acronym, struct_tree):
    areas_of_interest = {
        'SM-TH': ['AV', 'CL', 'MD', 'PO', 'PF', 'VAL', 'VPL', 'VPM', 'VM'],
    }
    
    reg_id = struct_tree.get_structures_by_acronym([region_acronym])[0]['id']
    if struct_tree.structure_descends_from(reg_id, 567):
        if struct_tree.structure_descends_from(reg_id, 315):
            return get_region_from_children(reg_id, 315, struct_tree)
        elif struct_tree.structure_descends_from(reg_id, 698):
            return 'OLF'
        elif struct_tree.structure_descends_from(reg_id, 1089):
            return get_region_from_children(reg_id, 1089, struct_tree)
        elif struct_tree.structure_descends_from(reg_id, 703):
            return get_region_from_children(reg_id, 703, struct_tree)
        elif struct_tree.structure_descends_from(reg_id, 477):
            return 'STR'
        elif struct_tree.structure_descends_from(reg_id, 803):
            return 'PAL'
        else:
            return 'unassigned'
    elif struct_tree.structure_descends_from(reg_id, 343):
        if struct_tree.structure_descends_from(reg_id, 1129):
            if region_acronym == 'RT':
                return 'RT-TH'
            elif region_acronym in areas_of_interest['SM-TH']:
                return 'SM-TH'
            else:
                return 'other-TH'
        elif struct_tree.structure_descends_from(reg_id, 1097):
            return 'HY'
        else:
            return get_region_from_children(reg_id, 343, struct_tree)
    else:
        return 'unassigned'

In [9]:
def add_parent_region_to_df(unit_info_df, struct_tree, annot):
    ## First, make sure all names in region column correspond to a CCF region (removes nan values) ##
    adj_regions = unit_info_df['region'].values.copy()
    for indi, rowi in unit_info_df.iterrows():
        try:
            str_info = struct_tree.get_structures_by_acronym([rowi.region])[0]
        except KeyError:
            if rowi.depth <= 0: # unit was placed above brain
                new_region_id = annot[rowi.CCF_AP, np.nonzero(annot[rowi.CCF_AP, :, rowi.CCF_ML])[0][0], rowi.CCF_ML]
                adj_regions[indi] = struct_tree.get_structures_by_id([new_region_id])[0]['acronym']
            else:
                Lind = rowi.CCF_ML
                while annot[rowi.CCF_AP, rowi.CCF_DV, Lind] == 0:
                    Lind -= 1
                new_region_id = struct_tree.get_structures_by_id(
                    [annot[rowi.CCF_AP, rowi.CCF_DV, Lind]])[0]['structure_id_path'][-1]
                adj_regions[indi] = struct_tree.get_structures_by_id([new_region_id])[0]['acronym']
    unit_info_df['adj_region'] = adj_regions
    
    ## Second, re-assign any non-grey matter areas to the closest region ##
    adj_regions = unit_info_df['adj_region'].values.copy()
    for indi, rowi in unit_info_df.iterrows():
        reg_id = struct_tree.get_structures_by_acronym([rowi.adj_region])[0]['id']
        if not struct_tree.structure_descends_from(reg_id, 8):
            adj_regions[indi] = find_closest_region(rowi, struct_tree, annot)
    unit_info_df['adj_region'] = adj_regions
    
    ## Finally, assign a parent region to each adjusted CCF region ##
    parent_regions = unit_info_df['adj_region'].values.copy()
    for indi, rowi in unit_info_df.iterrows():
        parent_regions[indi] = get_parent_region(rowi.adj_region, struct_tree)
    unit_info_df['parent_region'] = parent_regions
    
    return unit_info_df.drop('adj_region', axis=1)

#### Load Google excel log file to get metadata for experiments

In [10]:
_gc = gspread.service_account() # need a key file to access the account
_sh = _gc.open('Templeton-log_exp') # open the spreadsheet
# _sh = _gc.open('Zap_Zip-log_exp') # open the spreadsheet
_df = pd.DataFrame(_sh.sheet1.get()) # load the first worksheet
metadata = _df.T.set_index(0).T # put it in a nicely formatted dataframe

#### Load subjects.csv file to get CCF resolution

In [11]:
multisub_file = r"C:\Users\lesliec\OneDrive - Allen Institute\data\brain_states_subjects.csv"
subject_df = pd.read_csv(multisub_file, converters={'mouse': str}).astype({'analyze': bool})

### Load subjects

In [12]:
subjects = {
    '631037': {
        'awake-saline': r'F:\psi_exp\mouse631037\estim_2022-12-06_09-54-04\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse631037\urethane_2022-12-07_10-34-51\experiment1\recording1',
    },
    '654182': {
        'awake-saline': r'F:\psi_exp\mouse654182\estim_vis_2022-12-01_10-33-50\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse654182\urethane_vis_2022-12-02_11-02-25\experiment1\recording1',
    },
    '655956': {
        'awake-saline': r'F:\psi_exp\mouse655956\estim_2022-12-15_10-07-59\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse655956\urethane_2022-12-16_10-45-18\experiment1\recording1',
    },
    '657903': {
        'awake-psilocybin': r'F:\psi_exp\mouse657903\pilot_aw_psi_2023-01-13_12-18-22\experiment1\recording1',
    },
    '666193': {
        'saline': r'F:\psi_exp\mouse666193\pilot_aw_2023-02-15_11-44-11\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse666193\pilot_aw_psi_2023-02-16_10-55-48\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse666193\pilot_ur_2023-02-17_12-22-51\experiment1\recording1',
    }, 
    '666194': {
        'saline': r'F:\psi_exp\mouse666194\pilot_aw_2023-02-22_12-32-58\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse666194\pilot_aw_psi_2023-02-23_10-40-34\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse666194\pliot_ur_2023-02-24_11-19-43\experiment1\recording1',
    },
    '666196': {
        'saline': r'F:\psi_exp\mouse666196\pilot_aw_2023-03-15_12-29-06\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse666196\pilot_aw_psi_2023-03-16_10-21-29\experiment1\recording1',
    },
    '669118': {
        'saline': r'F:\psi_exp\mouse669118\pilot_aw_2023-03-23_12-14-39\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse669118\pilot_aw_psi_2023-03-24_09-55-33\experiment1\recording1',
    },
    '669117': {
        'saline': r'F:\psi_exp\mouse669117\pilot_aw_2023-03-29_11-09-15\experiment1\recording1',
        'psilocybin': r'F:\psi_exp\mouse669117\pilot_aw_psi_2023-03-30_11-37-07\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse669117\pilot_ur_2023-03-31_11-51-53\experiment1\recording1',
    },
    '673449': {
        'psilocybin1': r'F:\psi_exp\mouse673449\aw_psi_2023-04-19_11-23-26\experiment1\recording1',
        'psilocybin2': r'F:\psi_exp\mouse673449\aw_psi_d2_2023-04-20_10-05-31\experiment1\recording1',
        'saline': r'F:\psi_exp\mouse673449\aw_2023-04-21_09-28-23\experiment1\recording1',
    },
    '676726': {
        'psilocybin': r'F:\psi_exp\mouse676726\aw_psi_2023-05-03_11-08-22\experiment1\recording1',
        'awake-iso': r'F:\psi_exp\mouse676726\aw_iso_2023-05-04_11-02-16\experiment1\recording1',
    },
    '676727': {
        'psilocybin': r'F:\psi_exp\mouse676727\aw_psi_2023-05-10_09-49-12\experiment1\recording1',
        'awake-iso': r'F:\psi_exp\mouse676727\aw_iso_2023-05-11_09-44-46\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse676727\urethane_2023-05-12_11-35-38\experiment1\recording1',
    },
    '678912': {
        'psilocybin': r'F:\psi_exp\mouse678912\spont_aw_psi_2023-06-22_11-42-00\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse678912\urethane_2023-06-23_11-08-17\experiment1\recording1',
    },
    '678913': {
        'psilocybin': r'F:\psi_exp\mouse678913\spont_aw_psi_2023-06-29_12-49-40\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse678913\urethane_2023-06-30_10-56-33\experiment1\recording1',
    },
    ## awake-iso ##
    '551397': {
        'awake-iso': r'F:\ZZmanuscript_eLife\mouse551397\estim_vis_2021-02-11_10-45-23\experiment1\recording1',
    },
    '551399': {
        'awake-iso': r'F:\ZZmanuscript_eLife\mouse551399\estim_2021-01-28_13-59-09\experiment1\recording1',
    },
    '569062': {
        'awake-iso': r'F:\ZZmanuscript_eLife\mouse569062\estim_vis_2021-02-18_11-17-51\experiment1\recording1',
    },
    '569064': {
        'awake-iso': r'F:\ZZmanuscript_eLife\mouse569064\estim_vis_2021-04-08_10-28-24\experiment1\recording1',
    },
    '569068': {
        'awake-iso': r'F:\ZZmanuscript_eLife\mouse569068\estim_vis_2021-03-04_10-51-38\experiment1\recording1',
    },
    '569069': {
        'awake-iso': r'F:\ZZmanuscript_eLife\mouse569069\estim_vis2_2021-03-12_10-52-44\experiment1\recording1',
    },
    '569070': {
        'awake-iso': r'F:\ZZmanuscript_eLife\mouse569070\estim1_2021-04-01_10-27-33\experiment1\recording1',
    },
    '569073': {
        'awake-iso': r'F:\ZZmanuscript_eLife\mouse569073\estim_vis_2021-04-15_10-27-22\experiment1\recording1',
    },
    '571619': {
        'awake-iso': r'F:\ZZmanuscript_eLife\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
    },
    '635397': {
        'awake-iso': r'F:\psi_exp\mouse635397\estim_vis_2022-08-18_12-08-15\experiment1\recording1',
    },
    '654181': {
        'awake-iso': r'F:\psi_exp\mouse654181\estim_vis_2022-11-22_09-42-58\experiment1\recording1',
        'urethane': r'F:\psi_exp\mouse654181\urethane_vis_2022-11-23_08-30-16\experiment1\recording1',
    },
    ## urethane ##
    '638703': {
        'urethane': r'F:\psi_exp\mouse638703\urethane_estim_2022-10-14_12-25-20\experiment1\recording1',
    },
    '655955': {
        'urethane': r'F:\psi_exp\mouse655955\urethane_2022-12-14_10-38-00\experiment1\recording1',
    },
    '582386': {
        'urethane': r'F:\psi_exp\mouse582386\urethane_2021-07-15_11-36-58\experiment1\recording1',
    },
}

### Check trial quality for EEG signals

### Set parameters

In [13]:
overwrite_existing_files = False

event_window = [-2.0, 2.0]

apply_mask = True
apply_hpass = True
apply_lpass = True

### Process running signal and EEG

In [15]:
for mouse, explist in subjects.items():
    for exptype, dataloc in explist.items():
        print('{}: {}'.format(mouse, exptype))
        exp = EEGexp(dataloc, preprocess=False, make_stim_csv=False)
        
        ## Set file names ##
        running_file = os.path.join(exp.data_folder, 'running_signal.npy')
        running_ts_file = os.path.join(exp.data_folder, 'running_timestamps_master_clock.npy')
        evoked_folder = os.path.join(exp.data_folder, 'evoked_data')
        if not os.path.exists(evoked_folder):
            os.mkdir(evoked_folder)
        event_running_file = os.path.join(evoked_folder, 'event_running_speed.npy')
        event_running_ts_file = os.path.join(evoked_folder, 'event_running_times.npy')
        event_EEGtraces_file = os.path.join(evoked_folder, 'event_EEGtraces.npy')
        event_EEGtraces_ts_file = os.path.join(evoked_folder, 'event_EEGtraces_times.npy')

        ## Load stim log ##
        stim_log = pd.read_csv(exp.stimulus_log_file)
        all_event_times = stim_log['onset'].values
            
        ## Load running signal and get mean event speed ##
        if os.path.exists(running_file):
            run_signal = np.load(running_file)
            run_timestamps = np.load(running_ts_file)
        else:
            print('  Loading running from sync and saving...')
            run_signal, run_timestamps = exp.load_running()
            np.save(running_file, run_signal, allow_pickle=False)
            np.save(running_ts_file, run_timestamps, allow_pickle=False)
        if not os.path.exists(event_running_file) or overwrite_existing_files:
            print('  Getting event-related running...')
            rinds = np.arange(-int(-event_window[0] * 100), int(event_window[1] * 100))
            event_inds = np.array([find_nearest_ind(run_timestamps, x) for x in all_event_times])
            event_run_speed = run_signal[np.repeat([rinds], len(event_inds), axis=0).T + event_inds]
            event_run_times = rinds / 100
            ## Save ##
            np.save(event_running_file, event_run_speed, allow_pickle=False)
            np.save(event_running_ts_file, event_run_times, allow_pickle=False)
            ## Add speed to stim_log ##
            evinds = np.nonzero((event_run_times >= -0.5) & (event_run_times < 0.5))[0]
            mean_speed = np.mean(event_run_speed[evinds, :], axis=0)
            stim_log['mean_speed'] = mean_speed
            stim_log['resting_trial'] = stim_log['mean_speed'] == 0
            stim_log.to_csv(exp.stimulus_log_file, index=False)
            
        if np.any([True for x in exp.experiment_data if 'recording' in x]):
            ## Grab exp metadata from Templeton-log_exp ##
            exp_meta = metadata[(
                (metadata['mouse_name'].str.contains(mouse)) &
                (metadata['exp_name'].str.contains(os.path.basename(os.path.dirname(exp.experiment_folder))))
            )].squeeze()
            badchstr = exp_meta['EEG bad_channels'].replace(' ','')
            if (not os.path.exists(event_EEGtraces_file) or overwrite_existing_files) and (badchstr != 'all'):
                ## Load EEG data and preprocess ##
                print('  Loading EEG data...')
                datai, tsi = exp.load_eegdata()
                eeg_chs = np.arange(0, datai.shape[1])

                ## Mask estim artifact ##
                if apply_mask:
                    mask_samples = int(0.002 * exp.ephys_params['EEG']['sample_rate'])
                    for etime in stim_log.loc[stim_log['stim_type'] == 'biphasic', 'onset'].to_numpy():
                        val = find_nearest_ind(tsi, etime) - 2
                        datai[val:val+mask_samples, :] = datai[val:val-mask_samples:-1, :]

                ## Apply high-pass filter ##
                if apply_hpass:
                    hpb, hpa = signal.butter(3, 0.1/(exp.ephys_params['EEG']['sample_rate']/2), btype='highpass')
                    datai = signal.filtfilt(hpb, hpa, datai, axis=0)

                ## Get evoked traces ##
                print('  Getting EEG traces...')
                event_traces, event_ts = get_evoked_traces(
                    datai, tsi, all_event_times, -event_window[0], event_window[1], exp.ephys_params['EEG']['sample_rate'])

                ## Apply lowpass filter ##
                if apply_lpass:
                    lpb, lpa = signal.butter(3, 100/(exp.ephys_params['EEG']['sample_rate']/2), btype='low')
                    event_traces = signal.filtfilt(lpb, lpa, event_traces, axis=0)

                ## Save ##
                print('   ...saving {}.'.format(event_EEGtraces_file))
                np.save(event_EEGtraces_file, event_traces, allow_pickle=False)
                np.save(event_EEGtraces_ts_file, event_ts, allow_pickle=False)
            else:
                print('  Not creating EEG traces file, it already exists or all EEG chs are bad.')
        else:
            print('  No EEG in this recording.')

        print('')

631037: awake-saline
Experiment type: electrical stimulation
  Not creating EEG traces file, it already exists or all EEG chs are bad.

631037: urethane
Experiment type: electrical stimulation
  Not creating EEG traces file, it already exists or all EEG chs are bad.

654182: awake-saline
Experiment type: electrical and sensory stimulation
  Not creating EEG traces file, it already exists or all EEG chs are bad.

654182: urethane
Experiment type: electrical and sensory stimulation
  Not creating EEG traces file, it already exists or all EEG chs are bad.

655956: awake-saline
Experiment type: electrical stimulation
  Not creating EEG traces file, it already exists or all EEG chs are bad.

655956: urethane
Experiment type: electrical stimulation
  Not creating EEG traces file, it already exists or all EEG chs are bad.

657903: awake-psilocybin
Experiment type: electrical stimulation
  Not creating EEG traces file, it already exists or all EEG chs are bad.

666193: saline
Experiment type: 

### Process units

In [15]:
for mouse, explist in subjects.items():
    for exptype, dataloc in explist.items():
        print('{}: {}'.format(mouse, exptype))
        exp = EEGexp(dataloc, preprocess=False, make_stim_csv=False)
        
        probe_list = [x.replace('_sorted', '') for x in exp.experiment_data if 'probe' in x]
        if len(probe_list) == 0:
            print(' This experiment has no probe data, not making spike times files.\n')
            continue
        
        ## Set file names ##
        evoked_folder = os.path.join(exp.data_folder, 'evoked_data')
        if not os.path.exists(evoked_folder):
            os.mkdir(evoked_folder)
        unit_info_file = os.path.join(evoked_folder, 'all_units_info.csv')
        unit_allspiketimes_file = os.path.join(evoked_folder, 'units_allspikes.pkl')
        unit_eventspikes_file = os.path.join(evoked_folder, 'units_event_spikes.pkl')
        if overwrite_existing_files:
            pass # will overwrite all subjects' files
        else:
            if os.path.exists(unit_info_file):
                print('  {} already exists, skipping analysis.\n'.format(unit_info_file))
                continue

        ## Load stim log ##
        stim_log = pd.read_csv(exp.stimulus_log_file)
        all_event_times = stim_log['onset'].values
        
        ## Get probe info ##
        print('  Getting probe info...')
        probe_data = {}
        for pbi, probei in enumerate(probe_list):
            probe_data[probei] = {}
            ## Load probe_info.json ##
            with open(exp.ephys_params[probei]['probe_info']) as data_file:
                data = json.load(data_file)
            npx_allch = np.array(data['channel'])
            surface_ch = int(data['surface_channel'])
            allch_z = np.array(data['vertical_pos'])
            ref_mask = np.array(data['mask'])
            npx_chs = np.array([x for x in npx_allch if ref_mask[x] and x <= surface_ch])
            probe_data[probei]['ch_depths'] = allch_z[surface_ch] - allch_z
            
            ## Select units and get peak chs ##
            select_units, peak_chs, unit_metrics = exp.get_probe_units(probei)
            ## Sort units ##
            probe_data[probei]['units'] = select_units[np.squeeze(np.argsort(peak_chs))]
            probe_data[probei]['chs'] = peak_chs[np.squeeze(np.argsort(peak_chs))]
            probe_data[probei]['duration'] = unit_metrics.duration.values[np.squeeze(np.argsort(peak_chs))]
            
            ## Load spike times and cluster ids ##
            probe_data[probei]['spike_times'] = np.load(exp.ephys_params[probei]['spike_times'])
            probe_data[probei]['spike_clusters'] = np.load(exp.ephys_params[probei]['spike_clusters'])
            
            if 'area_ch' in data.keys():
                probe_data[probei]['areas'] = unit_metrics.area.values[np.squeeze(np.argsort(peak_chs))]
                probe_data[probei]['CCF_coords'] = unit_metrics.ccf_coord.values[np.squeeze(np.argsort(peak_chs))]
                
        ## Get unit info, spikes and event-spikes, then save files ##
        print('  Getting spike times...')
        start = time.time()
        all_units_info = []
        unit_allspiketimes = {}
        unit_eventspikestimes = {'event_window': event_window, 'event_spikes': {}, 'event_bursts': {}}
        for probei, pdata in probe_data.items():
            for unitind, uniti in enumerate(pdata['units']):
                unit_name = probei[-1] + str(uniti)
                spikesi = np.squeeze(pdata['spike_times'][pdata['spike_clusters'] == uniti])
                if spikesi.size < 50:
                    continue
                unit_allspiketimes[unit_name] = {}
                
                ## Gather unit info ##
                if 'areas' in pdata.keys():
                    unit_region = pdata['areas'][unitind]
                    unit_coords = unit_coords = [
                        int(x) for x in pdata['CCF_coords'][unitind].replace('[','').replace(']','').replace(' ','').split(',')
                    ]
                else:
                    unit_region = 'none'
                    unit_coords = [-1, -1, -1]
                all_units_info.append([
                    unit_name, probei, pdata['chs'][unitind], pdata['ch_depths'][pdata['chs'][unitind]],
                    pdata['duration'][unitind], unit_region, unit_coords[0], unit_coords[1], unit_coords[2]
                ])

                ## Get all and event spike times ##
                unit_allspiketimes[unit_name]['spikes'] = spikesi
                burstsi, burst_counts = find_bursts_indunit(spikesi)
                unit_allspiketimes[unit_name]['bursts'] = burstsi
                unit_allspiketimes[unit_name]['burst_counts'] = burst_counts
                event_raster = []
                burst_raster = []
                burst_count_raster = []
                for eventi in all_event_times:
                    spikeinds = np.nonzero((spikesi >= eventi + event_window[0]) & (spikesi <= eventi + event_window[1]))[0]
                    event_raster.append(spikesi[spikeinds] - eventi)
                    burstinds = np.nonzero((burstsi >= eventi + event_window[0]) & (burstsi <= eventi + event_window[1]))[0]
                    burst_raster.append(burstsi[burstinds] - eventi)
                    burst_count_raster.append(burst_counts[burstinds])
                unit_eventspikestimes['event_spikes'][unit_name] = event_raster
                unit_eventspikestimes['event_bursts'][unit_name] = {'times': burst_raster, 'counts': burst_count_raster}

        ## Save the data files to mouse's recordingX\evoked_data folder ##
        all_units_info_df = pd.DataFrame(
            all_units_info,
            columns=['unit_id', 'probe', 'peak_ch', 'depth', 'spike_duration', 'region', 'CCF_AP', 'CCF_DV', 'CCF_ML']
        )
        
        ## Add parent region column ##
        if len(np.unique(all_units_info_df['region'].values.astype(str))) > 1:
            print('  Adding parent region...')
#             sub_CCF_res = subject_df[subject_df['mouse'] == mouse]['CCF_res'].iloc[0]
            sub_CCF_res = 25
            mcc = MouseConnectivityCache(resolution=sub_CCF_res)
            str_tree = mcc.get_structure_tree()
            annot, annot_info = mcc.get_annotation_volume()
            all_units_info_df = add_parent_region_to_df(all_units_info_df, str_tree, annot)
        
        all_units_info_df.to_csv(unit_info_file, index=False)
        pickle.dump(unit_allspiketimes, open(unit_allspiketimes_file, 'wb'))
        pickle.dump(unit_eventspikestimes, open(unit_eventspikes_file, 'wb'))

        end = time.time()
        print('  Time to get unit spike times and save: {:.2f} min\n'.format((end-start)/60))
        ## After each subject, delete common variables ##
        del stim_log, probe_data, all_event_times, all_units_info, all_units_info_df, unit_allspiketimes, unit_eventspikestimes

678912: psilocybin
Experiment type: sensory stimulation
  Getting probe info...
  Getting spike times...
  Adding parent region...
  Time to get unit spike times and save: 1.08 min

678912: urethane
Experiment type: electrical and sensory stimulation
  Getting probe info...
  Getting spike times...
  Adding parent region...
  Time to get unit spike times and save: 1.57 min

678913: psilocybin
Experiment type: sensory stimulation
  Getting probe info...
  Getting spike times...
  Adding parent region...
  Time to get unit spike times and save: 0.64 min

678913: urethane
Experiment type: electrical and sensory stimulation
  Getting probe info...
  Getting spike times...
  Adding parent region...
  Time to get unit spike times and save: 0.98 min

